In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

from tensorflow_graph_in_jupyter import show_graph
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tfgraphviz as tfg

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [6]:
reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.71450037
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555717
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.54363626
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [ ]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "./my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./my_model_final.ckpt")

In [7]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401655],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269557],
       [-0.66145295],
       [-0.63752794]], dtype=float32)

In [8]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    best_theta_restored = theta.eval()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [9]:
np.allclose(best_theta, best_theta_restored)

True

In [10]:
saver = tf.train.Saver({"weights": theta})

In [11]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [12]:
np.allclose(best_theta, best_theta_restored)

True

In [ ]:
show_graph(tf.get_default_graph())

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [14]:
%tensorboard --logdir ./

Reusing TensorBoard on port 6006 (pid 2564), started 0:01:16 ago. (Use '!kill 2564' to kill it.)